In [1]:
import uuid
import random
import math

import dask.dataframe as dd
from glob import glob
import numpy as np
import pandas as pd

from dask.diagnostics import ProgressBar

pbar = ProgressBar()
pbar.register()

/home/zibg/vi-projects/others/lakey-service/data_generator/.venv/lib/python3.6/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


# Generate fake devices with geolocation

## Configuration

In [2]:
number_of_devices = 10000
number_of_cores = 8

# real data
weather_real_path = '../data/weather_real/*/*/*.parquet'

# save to
fake_devices_with_geolocation_path = '../data/device_geolocations_fake/{}_devices_with_geolocation.parquet'.format(str(number_of_devices))

In [3]:
area_matrix = [ # lat, lng of the center, radius in km, probability
    [48.24, 11.48, 35, 0.07],
    [51.17, 7.01, 40, 0.24],
    [50.31, 8.86, 50, 0.11],
    [49.65, 7.39, 15, 0.06],
    [48.54, 8.73, 10, 0.01],
    [49.46, 11.23, 20, 0.06],
    [52.73, 13.36, 30, 0.1],
    [52.27, 11.68, 20, 0.07],
    [52.68, 9.02, 50, 0.21],
    [53.69, 10.13, 25, 0.05],
    [51.18, 13.69, 30, 0.02]]

## Generate device ids and geolocations

In [4]:
location_index = np.random.choice(
    [x for x in range(len(area_matrix))],
    number_of_devices,
    p=[loc[3] for loc in area_matrix])

In [5]:
# 1 degree = ~111km
devices = pd.DataFrame({
    'gateway_uuid': [str(uuid.uuid4()) for x in range(number_of_devices)],
    'geo_lat': [round(area_matrix[x][0] + random.uniform(-1 * area_matrix[x][3]/111, area_matrix[x][3]/111), 4) for x in location_index],
    'geo_lng': [round(area_matrix[x][1] + random.uniform(-1 * area_matrix[x][3]/111, area_matrix[x][3]/111), 4) for x in location_index]
})

In [6]:
devices.head()

,gateway_uuid,geo_lat,geo_lng
0,cc0e549d-7082-4790-9770-9af47b3d06db,52.2699,11.6794
1,da89839e-57f2-4d3d-bd03-2cdadaaf95d7,52.6814,9.0188
2,61cf99bb-24a3-4262-b5ee-a8895d4da322,49.6495,7.3896
3,a604ac31-4ff2-4ad0-8f1f-d7c9dbe5197a,53.6900,10.1300
4,83eeb51f-ba75-4951-88a1-0d602e28e779,52.2702,11.6805


## Find real zip codes for geolocations (based on weather data)

In [7]:
weather_raw = dd.read_parquet(glob(weather_real_path))

In [8]:
locations = weather_raw[['zip_code', 'obs_geo_lat', 'obs_geo_lng']]
locations['zip_code'] = locations['zip_code'].astype(str)
locations['obs_geo_lat'] = locations['obs_geo_lat'].astype(float)
locations['obs_geo_lng'] = locations['obs_geo_lng'].astype(float)
locations = locations.groupby(['zip_code']).first()
locations = locations.reset_index()
locations = locations.compute()

[########################################] | 100% Completed |  6.1s


In [9]:
def nearest_zipcode(y):
    lat = float(y.geo_lat)
    lng = float(y.geo_lng)
    def distance(x):
        lat1 = float(lat)
        lon1 = float(lng)
        lat2 = float(x.obs_geo_lat)
        lon2 = float(x.obs_geo_lng)
        p = math.pi / 180
        a = (
            0.5 - math.cos((lat2 - lat1) * p) / 2 +
            math.cos(lat1 * p) * math.cos(lat2 * p) * (1 - math.cos((lon2 - lon1) * p)) / 2)

        return round(2 * 6371 * 1000 * math.asin(math.sqrt(a))) # distance in meters

    locations_around = (locations
        .where(locations.obs_geo_lat > lat - 0.5)
        .where(locations.obs_geo_lat < lat + 0.5)
        .where(locations.obs_geo_lng > lng - 0.5)
        .where(locations.obs_geo_lng < lng + 0.5)
        .dropna()
        .reset_index())
    
    locations_around['distance'] = locations_around.apply(distance, axis=1)
    return locations_around.loc[locations_around['distance'].idxmin()]['zip_code']

In [10]:
ddata = dd.from_pandas(devices, npartitions=number_of_cores)

In [11]:
devices['zip_code'] = ddata.apply(lambda x: nearest_zipcode(x), meta=('x', str), axis=1).compute()

[########################################] | 100% Completed |  4min 32.3s


In [12]:
devices.head()

,gateway_uuid,geo_lat,geo_lng,zip_code
0,cc0e549d-7082-4790-9770-9af47b3d06db,52.2699,11.6794,39326
1,da89839e-57f2-4d3d-bd03-2cdadaaf95d7,52.6814,9.0188,31608
2,61cf99bb-24a3-4262-b5ee-a8895d4da322,49.6495,7.3896,55743
3,a604ac31-4ff2-4ad0-8f1f-d7c9dbe5197a,53.6900,10.1300,22397
4,83eeb51f-ba75-4951-88a1-0d602e28e779,52.2702,11.6805,39326


## Save to parquet file

In [13]:
devices.to_parquet(fake_devices_with_geolocation_path)